In [1]:
#!/usr/bin/env python
# ------------------------------------------------------------------------------------------------------%
# Created by "Chia E Tungom" 11/15/2020                                                                 %
#       Email:      chemago99@yahoo.com                                                                 %
#       Github:     https://github.com/tungom                                                           %
#-------------------------------------------------------------------------------------------------------%

In [213]:
import random
import math
import numpy as np

In [222]:
# Constraint Problems 

# define function
def F1(x):
    
    """where the bounds are [(0, 100), (0,150)] 
    Airline revenue management 
     """
    return 617*x[0] + 238*x[1]

#define constraints
def C1(x, n):
    """ takes values and return tuple. (fitnees, [list of constraint values], number of constraints voilated)
    x: position 
    n: total number of constraints"""
    
    c1 = 166 - (x[0] + x[1]) 
     
    
    cs = [c1]
    
    if n != len(cs):
        print("number of constraints More OR Less")
    nv = 0
    pen = 0
    for i in cs:
        if  i >= 0 :
            nv += 1
            
    if nv > 0:
        F = F1(x) + sum(cs)   # voilated fitness calculation
    else:
        F = F1(x)
    return F, nv

# second constraint technique
def C2(x, n):
    """ takes values and return tuple. (fitnees, [list of constraint values], number of constraints voilated)
    x: position 
    n: total number of constraints"""
    
    #maxp = [100 for i in range(n)]
    
    c1 = 166 - (x[0] + x[1] )
     
    cs = [c1]
    
    if n != len(cs):
        print("number of constraints More OR Less")
    nv = 0
    for i in cs:
        if  i < 0 :
            nv += 1
            
    if nv > 0:
        F = sum([i/50 for i in cs])   # voilated fitness calculation
    else:
        F = F1(x)
    return F, nv

def C3(x, n):
    """ takes values and return tuple. (fitnees, [list of constraint values], number of constraints voilated)
    x: position 
    n: total number of constraints"""
    
    #maxp = [100 for i in range(n)]
    
    c1 = 166 - (x[0] + x[1])
    cs = [c1]
    
    if n != len(cs):
        print("number of constraints More OR Less")
    nv = 0
    pen = sum(cs)
    for i in cs:
        if  i < 0 :
            # print("HERE ", i, x)
            nv += 1
            
    if nv > 0:
        # F = sum([i/50 for i in cs])   # voilated fitness calculation
        F = F1(x)/(1+nv)
    else:
        F = F1(x)
    return F, nv

In [223]:
# Initializing position and Velocity

def InitializeX(N, D, BDS):
    """ N: population size (interger number)
        D: dimension (interger number)
        BDS: bounds (list with tuples e.g [(a,b),...,(c,d)]). List must be same length with D"""

    return [[random.uniform(BDS[i][0], BDS[i][1]) for i in range(D)] for i in range(N)]

def InitializeV(N, D, BDS):
    """ N: population size (interger number)
        D: dimension (interger number)
        BDS: bounds (list with tuples e.g [(a,b),...,(c,d)]). List must be same length with D"""

    return [[random.uniform(BDS[i][0], BDS[i][1]) for i in range(D)] for i in range(N)]

In [224]:
def PGbest(function, nc, x, pbest, gbest):
    
    """ function: function to be optimized
        nc: number of constraints
        x: current position of swarm members
        pbest: list of personal best positions [([position],(fitness, voilations)), ..., ([position],(fitness, voilations)) ]
        gbest: position of best individual """
    
    fx = [(i, function(i, nc)) for i in x]           #gives the position and fitness value [(position, (fitness, voilations) )]
    Fis = 0  # flag for feasible solution
    
    for i in range(len(x)):
        
        if fx[i][1][1] == 0 and pbest[i][1][1] == 0:    # compare feasible solutions
            Fis += 1
            
            if fx[i][1][0] > pbest[i][1][0]:
                pbest[i] = fx[i]
                
            if pbest[i][1][0] > gbest[1][0]:
                gbest = pbest[i]
                
        elif fx[i][1][1] == 0 and pbest[i][1][1] > 0:  # replace pbest if infeasible with new feasible solution
            pbest[i] = fx[i]
            
            if Fis == 0 and pbest[i][1][0] > gbest[1][0]:
                gbest = pbest[i]
                
        elif fx[i][1][1] > 0 and pbest[i][1][1] > 0: # compare two infeasible solutions 
            if fx[i][1][0] > pbest[i][1][0]:
                pbest[i] = fx[i]
                
            if Fis == 0 and pbest[i][1][0] > gbest[1][0]:
                gbest = pbest[i]
    
    return pbest, gbest

In [225]:
def updateV(vel, x, pbest, gbest, probT, w, c1, c2):
    
    """vel: A list of list with velocities of particles [[velocity],...,[velocity]] 
       x: current position of swarm members
       pbest: list of personal best positions [([position],(fitness, voilations)), ..., ([position],(fitness, voilations)) ]
       gbest: position of best individual 
       probT: turbulance probability
       w: inertia weight in the range [0.3, 0.9]
       c1 and c2: acceleration coefficients in the range [0, 4.0]"""
    
    v = []
    
    for i in range(len(x)):
        v.append([])
        tt = 0
        for j in range(len(x[0])):
            r1 = random.random()
            r2 = random.random()
            
            # Turbulance
            if probT > random.random():
                tt +=1
                v[i].append( round( (random.uniform(0.1, 0.6)*vel[i][j]) + 
                            (random.uniform(1.5, 3.5) * r1*(pbest[i][0][j] - x[i][j])) + 
                            (random.uniform(1.5, 3.5)* r2* (gbest[0][j] - x[i][j]))  + random.uniform(0,1) ) )
            else:
                v[i].append(  round( (random.uniform(0.1, 0.3)*vel[i][j]) + 
                            (random.uniform(0.5, 2.5) * r1*(pbest[i][0][j] - x[i][j])) + 
                            (random.uniform(0.5, 2.5)* r2* (gbest[0][j] - x[i][j])) ) )
        #if tt > 0 :
        #    print(tt, ": Turbulance recorded for Particle ", i)
            
    return v


def updateX(x, vel, bound):
    """vel: A list of list with velocities of particles [[velocity],...,[velocity]] 
       x: current position of swarm members
       bound: boundary constraints [(a,b),...,(c,d)]"""
    NX = []
    for i in range(len(x)):
        NX.append([])
        
        for j in range(len(x[0])):
            NX[i].append( round(x[i][j] + vel[i][j] ) )
            #print(x)
            if NX[i][j] < bound[j][0] or NX[i][j] > bound[j][1]:
                NX[i][j] = round(random.uniform(bound[j][0],bound[j][1]) )
            
    return NX

In [236]:
def CPSO(function, nc, dimension, bounds, population, runs):
    """ funtion: function to be optimized
        nc: number of constraints
        dimension: dimension of problem
        bounds: upper and lower bound for each dimension
        population: number of particles in the swarm
        runs: number of interations """
    
    positions = InitializeX(population, dimension, bounds)
    velocities = InitializeV(population, dimension, bounds)
    
    c1 = 1.5
    c2 = 0.5
    w = 0.6
    
    i = 0
    
    while i < runs : 
        # Turbulance Operator
        temp =  i /runs
        ProbT = (temp**1.7) - (2.0*temp) + 1 
        #print(ProbTurb)
    
        if i == 0:
            pbest = [(i,function(i, nc)) for i in positions]
            pbest, gbest = PGbest(function, nc, positions, pbest, pbest[0])
        
        velocities = updateV(velocities, positions, pbest, gbest, ProbT, w, c1, c2)
        positions = updateX(positions, velocities, bounds)
        pbest, gbest = PGbest(function, nc, positions, pbest, gbest)
        
        if i%int(runs*0.05) == 0 :
            print("Genaration: ", i, " Fitness: ", gbest[1][0], " Real Fitness: ", F1(gbest[0]),
                  " Voilated Constraints: ", gbest[1][1])
        
        i += 1
        
        #print(min ([function(i) for i in X]))
        
    return gbest

In [238]:
nc = 1
D = 2
bounds = [(0,100),(0,150)]
pop = 50
Iter = 50

OR = CPSO(C3, nc, D, bounds, pop, Iter)

Genaration:  0  Fitness:  74764.40793952784  Real Fitness:  74764.40793952784  Voilated Constraints:  0
Genaration:  2  Fitness:  74940  Real Fitness:  74940  Voilated Constraints:  0
Genaration:  4  Fitness:  76791  Real Fitness:  76791  Voilated Constraints:  0
Genaration:  6  Fitness:  76791  Real Fitness:  76791  Voilated Constraints:  0
Genaration:  8  Fitness:  76791  Real Fitness:  76791  Voilated Constraints:  0
Genaration:  10  Fitness:  77170  Real Fitness:  77170  Voilated Constraints:  0
Genaration:  12  Fitness:  77170  Real Fitness:  77170  Voilated Constraints:  0
Genaration:  14  Fitness:  77170  Real Fitness:  77170  Voilated Constraints:  0
Genaration:  16  Fitness:  77170  Real Fitness:  77170  Voilated Constraints:  0
Genaration:  18  Fitness:  77170  Real Fitness:  77170  Voilated Constraints:  0
Genaration:  20  Fitness:  77170  Real Fitness:  77170  Voilated Constraints:  0
Genaration:  22  Fitness:  77170  Real Fitness:  77170  Voilated Constraints:  0
Genaratio

In [161]:
OR

([100, 66], (77408, 0))

In [162]:
F1(OR[0])

77408

In [163]:
x = OR[0]
c1 = 166 - (x[0] + x[1])
c1

0